In [6]:
from langchain_core.prompts import PromptTemplate

common_prompt_template = PromptTemplate.from_template(
    """You are an expert in {dialect}. Your job is to read and understand the following [Database Schema] description, along with any [Reference Information], and then use your knowledge of {dialect} to generate an SQL statement that answers the [User Question]. Pay attention to the [Database Schema], only use tables and columns that are in the [Database Schema]. Avoid using any other tables or columns that are not in the [Database Schema].

[User Question]
{user_question}

[Database Schema]
{schema}

[Reference Information]
{example_rows}

ONLY OUTPUT THE SQL STATEMENT, NO OTHER TEXT.
"""
)
xiyan_en_prompt_template = PromptTemplate.from_template(
    """You are an expert in {dialect}. Your job is to read and understand the following 【Schema】 description, along with any 【Evidence】, and then use your knowledge of {dialect} to generate an SQL statement that answers the 【Question】.

【Question】
{user_question}

【Schema】
{schema}

【Evidence】
{example_rows}

【Question】
{user_question}
```sql
"""
)
xiyan_cn_prompt_template = PromptTemplate.from_template(
    """你是一名{dialect}专家，现在需要阅读并理解下面的【数据库schema】描述，以及可能用到的【参考信息】，并运用{dialect}知识生成sql语句回答【用户问题】。
【用户问题】
{user_question}

【数据库schema】
{schema}

【参考信息】
{example_rows}

【用户问题】
{user_question}

```sql"""
)

In [7]:
import os
from dotenv import load_dotenv

load_dotenv()


llama_model_base_config = {
    "model_name": "llama3.2:3b",
    "model_config_init": {
        "temperature": 0.1,
    },
    "prompt_template": common_prompt_template,
    "inference_type": "ollama",
    "use_mschema": False,
}
xiyan7b16_model_base_config = {
    "model_name": "hf.co/mradermacher/XiYanSQL-QwenCoder-7B-2504-GGUF:F16",
    "model_config_init": {"temperature": 0.1, "top_p": 0.8},
    "prompt_template": xiyan_en_prompt_template,
    "inference_type": "ollama",
}

model_config = {
    "llama32_3b_base": {
        **llama_model_base_config,
        "use_example_rows": False,
        "retry_times": 0,
    },
    "llama32_3b_w_ex": {
        **llama_model_base_config,
        "use_example_rows": True,
        "retry_times": 0,
    },
    "llama32_3b_w_re": {
        **llama_model_base_config,
        "use_example_rows": False,
        "retry_times": 10,
    },
    "llama32_3b_all": {
        **llama_model_base_config,
        "use_example_rows": True,
        "retry_times": 10,
    },
    "llama31_8b_all": {
        **llama_model_base_config,
        "model_name": "llama3.1:8b",
        "use_example_rows": True,
        "retry_times": 10,
    },
    # "llama31_8b_all": {
    #     "prompt_template": common_prompt_template,
    #     "use_mschema": False,
    #     "model_name": "meta-llama/Llama-3.1-8B",
    #     "inference_type": "vllm",
    #     "use_example_rows": True,
    #     "retry_times": 10,
    #     "model_config_init": {
    #         "temperature": 0.1,
    #         "vllm_kwargs": {
    #             "hf_token": os.environ["HF_TOKEN"],
    #             "max_model_len": 20000,
    #         },
    #     },
    # },
    "xiyansql_7b_16_base": {
        **xiyan7b16_model_base_config,
        "use_mschema": False,
        "use_example_rows": False,
        "retry_times": 0,
    },
    "xiyansql_7b_16_w_ex": {
        **xiyan7b16_model_base_config,
        "use_mschema": False,
        "use_example_rows": True,
        "retry_times": 0,
    },
    "xiyansql_7b_16_w_re": {
        **xiyan7b16_model_base_config,
        "use_mschema": False,
        "use_example_rows": False,
        "retry_times": 10,
    },
    "xiyansql_7b_16_all": {
        **xiyan7b16_model_base_config,
        "use_mschema": False,
        "use_example_rows": True,
        "retry_times": 10,
    },
    "xiyansql_7b_8_all": {
        **xiyan7b16_model_base_config,
        "model_name": "hf.co/mradermacher/XiYanSQL-QwenCoder-7B-2504-GGUF:Q8_0",
        "use_mschema": False,
        "use_example_rows": True,
        "retry_times": 10,
    },
}

In [8]:
from langchain_ollama import ChatOllama
from langchain_community.llms import VLLM
import pandas as pd
from tqdm.notebook import tqdm
from sqlalchemy import create_engine
from sqlalchemy.sql import text
from langchain_core.prompt_values import PromptValue
from langchain_openai import ChatOpenAI

tqdm.pandas()


class ModelCreater:
    def __init__(self):
        self.llm = None
        self.model_name = None
        self.infer_type = None

    def create_model(self, model_config: dict):
        if (
            self.llm
            and self.model_name == model_config["model_name"]
            and self.infer_type == model_config["inference_type"]
        ):
            return self.llm
        else:
            # shutdown created vllm
            if self.infer_type == "vllm":
                pass

            if model_config["inference_type"] == "ollama":
                self.llm = ChatOllama(
                    model=model_config["model_name"],
                    **model_config["model_config_init"],
                )
            elif model_config["inference_type"] == "vllm":
                self.llm = VLLM(
                    model=model_config["model_name"],
                    trust_remote_code=True,
                    **model_config["model_config_init"],
                )
            elif model_config["inference_type"] == "vllm-openai":
                self.llm = ChatOpenAI(
                    model=model_config["model_name"],
                    base_url="http://127.0.0.1:8000/",
                    api_key="EMPTY",
                    **model_config["model_config_init"],
                )
            else:
                raise ValueError(
                    f"Inference type {model_config['inference_type']} not supported"
                )
        self.model_name = model_config["model_name"]
        self.infer_type = model_config["inference_type"]
        return self.llm


model_creater = ModelCreater()
retry_prompt_template = PromptTemplate.from_template(
    """Your query is invalid. Carefully read the table schemas and the user question, and then regenerate a new query.
    
User question:
{user_question}

Table schemas:
{schema}

Here is your generated query:
{invalid_query}

And here is the error:
{error_message}

ONLY OUTPUT THE SQL STATEMENT, NO OTHER TEXT.
"""
)


def check_sql_valid(sql: str, db_path: str):
    engine = create_engine(f"sqlite:///{db_path}")
    try:
        with engine.connect() as conn:
            conn.execute(text("EXPLAIN " + sql))
    except Exception as e:
        return False, repr(e)
    return True, ""


def gen_sql(llm, prompt: PromptValue):
    message = llm.invoke(prompt)
    if not isinstance(message, str):
        message = message.content
    message = (
        message.strip()
        .replace("```sql", "")
        .replace("```", "")
        .replace("\n", "")
        .replace("\t", "")
    )
    return message


def benchmark_model(
    model: str,
    dialect: str,
    input_file_path: str,
    output_file_path: str,
    db_root_dir: str,
):
    config = model_config[model]
    llm = model_creater.create_model(config)
    prompt_template = config["prompt_template"]

    def generate_sql(
        user_question: str,
        schema: str,
        dialect: str,
        db_id: str,
        example_rows: list[str] | None = None,
    ):
        prompt_msg = prompt_template.invoke(
            {
                "user_question": user_question,
                "schema": schema,
                "dialect": dialect,
                "example_rows": example_rows if example_rows else "None",
            }
        )
        message = gen_sql(llm, prompt_msg)

        if config["retry_times"] > 0:
            db_path = f"{db_root_dir}/{db_id}/{db_id}.sqlite"
            is_valid, err_msg = check_sql_valid(message, db_path)

            i = 0
            while not is_valid and i < config["retry_times"]:
                retry_prompt_msg = retry_prompt_template.invoke(
                    {
                        "user_question": user_question,
                        "schema": schema,
                        "invalid_query": message,
                        "error_message": err_msg,
                    }
                )
                # print(
                #     f"({i + 1}/{config['retry_times']}) Retry prompt: {retry_prompt_msg}"
                # )
                message = gen_sql(llm, retry_prompt_msg)
                is_valid, err_msg = check_sql_valid(message, db_path)
                i += 1
        return message

    df = pd.read_csv(input_file_path)
    inference_df = df.assign(
        answer=lambda df_: df_.progress_apply(
            lambda row: generate_sql(
                user_question=row.question,
                schema=row.schemas if not config["use_mschema"] else row.mschemas,
                dialect=dialect,
                db_id=row.db_id,
                example_rows=row.example_rows if config["use_example_rows"] else None,
            ),
            axis=1,
        )
    )
    with open(output_file_path, "w+") as f:
        answer_list = inference_df["answer"].tolist()
        f.write("\n".join(answer_list))

    return inference_df

In [9]:
test_list = [
    # ("llama32_3b_base", "dev"),
    # ("llama32_3b_w_ex", "dev"),
    # ("llama32_3b_w_re", "dev"),
    # ("llama32_3b_all", "dev"),
    # ("xiyansql_7b_16_base", "dev"),
    # ("xiyansql_7b_16_w_ex", "dev"),
    # ("xiyansql_7b_16_w_re", "dev"),
    # ("xiyansql_7b_16_all", "dev"),
    # ("llama32_3b_all", "test"),
    ("llama31_8b_all", "dev"),
    # ("xiyansql_7b_16_all", "test"),
    # ("xiyansql_7b_8_all", "test"),
]

In [10]:
for model_name, dataset in test_list:
    dialect = "sqlite"
    input_file_path = f"prepare_data/{dataset}_input.csv"
    output_file_path = f"inference_data/{model_name}_{dataset}_inf.txt"
    db_root_dir = (
        "spider_data/database" if dataset == "dev" else "spider_data/test_database"
    )
    print(f"Benchmarking {model_name}, {dataset} dataset")
    inference_df = benchmark_model(
        model_name, dialect, input_file_path, output_file_path, db_root_dir
    )

Benchmarking llama31_8b_all, dev dataset


  0%|          | 0/1034 [00:00<?, ?it/s]